In [5]:
import requests
import pandas as pd
import time
import os
import sys

# Add the 'scripts' folder to the Python path
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from discord_notifier import send_discord_message

# Ensure the directory exists
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to retrieve PUUIDs from summoner_details CSV
def retrieve_puuids(csv_file='../data/processed/summoner_details.csv', increase_pool=False):
    # Read the summoner details file
    df_summoner = pd.read_csv(csv_file)

    # If we are increasing the pool, retrieve all PUUIDs
    if increase_pool:
        return df_summoner['puuid'].tolist()

    # Otherwise, filter players who don't have match IDs in match_ids.csv
    if os.path.exists('../data/processed/match_ids.csv'):
        df_match_ids = pd.read_csv('../data/processed/match_ids.csv')
        puuids_with_matches = df_match_ids['puuid'].unique()
        puuids_to_fetch = df_summoner[~df_summoner['puuid'].isin(puuids_with_matches)]['puuid'].tolist()
    else:
        puuids_to_fetch = df_summoner['puuid'].tolist()

    return puuids_to_fetch

# Function to append match ID-PUUID pairs to match_ids CSV
def insert_match_ids(match_id_puuid_pairs, csv_file='../data/processed/match_ids.csv'):
    # Check if the file exists
    if os.path.exists(csv_file):
        df_match_ids = pd.read_csv(csv_file)
    else:
        # If it doesn't exist, create an empty DataFrame
        df_match_ids = pd.DataFrame(columns=['match_id', 'puuid'])

    # Convert the match_id_puuid_pairs into a DataFrame
    new_data = pd.DataFrame(match_id_puuid_pairs, columns=['match_id', 'puuid'])

    # Append new data and remove duplicates
    df_match_ids = pd.concat([df_match_ids, new_data]).drop_duplicates()

    # Save the updated DataFrame back to CSV
    df_match_ids.to_csv(csv_file, index=False)
    print(f"Inserted {len(new_data)} match ID-PUUID pairs into {csv_file}.")

# Function to pull match IDs from the Riot API
def pull_match_ids(api_key, puuid, base_url, matches_per_player):
    url = f"{base_url}{puuid}/ids?start=0&count={matches_per_player}&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return [(match_id, puuid) for match_id in response.json()]
    else:
        print(f"Error {response.status_code} fetching matches for {puuid}: {response.text}")
        return []

# Function to manage API rate limits
def manage_api_rate_limit():
    time.sleep(1.3)  # Adjust based on the Riot Games API rate limits

# Main function to pull and store match IDs
def main(increase_pool=False, matches_per_player=1):
    api_key = "RGAPI-b8762239-29e3-48eb-a6c7-5b9198da9aa1"
    base_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/"
    players = retrieve_puuids(increase_pool=increase_pool)
    batch_size = 1000  # Set the threshold for batch processing
    accumulated_pairs = []

    try:
        count = 1
        total = len(players)
        for puuid in players:
            print(f"Fetching: {count} out of {total} - PUUID: {puuid}")
            match_id_puuid_pairs = pull_match_ids(api_key, puuid, base_url, matches_per_player)
            if match_id_puuid_pairs:
                accumulated_pairs.extend(match_id_puuid_pairs)
                if len(accumulated_pairs) >= batch_size:
                    insert_match_ids(accumulated_pairs)
                    accumulated_pairs = []  # Reset the list after insertion
            manage_api_rate_limit()
            count += 1
        
        # Insert any remaining pairs that didn't reach the batch size
        if accumulated_pairs:
            insert_match_ids(accumulated_pairs)

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    try:
        main(increase_pool=True, matches_per_player=59)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        send_discord_message()

Fetching: 1 out of 87 - PUUID: rCj7vrcJ3AT8I81G-tn-mi7LmIRuosizW7drTrcnXtQUmy0hAdGc576W1j3n9F2WCIlnmusl4M4_wQ
Fetching: 2 out of 87 - PUUID: AnNB63AHxR63qY1ftYURyMzqf7dXu86SqeKdmuF__e7Xsoj74gYRWy68spmPQJxvrFcQb7m515XzYA
Fetching: 3 out of 87 - PUUID: qbkES1f5QwZIdpQ6jLTnK7vxHjAsCZV4iGYtLuweumLQd4_jYqHc6o_o9CHf2KHMizZsqDExT943hg
Fetching: 4 out of 87 - PUUID: dWT1LzI9J7wO8DrhhxJtB30S2hdgw3WBVif6PENwNt1-nVQwfD6fzxLjUJqkO2O4PfDoDE_ew3_qew
Fetching: 5 out of 87 - PUUID: NEFjCDWzn6tFYYhLUoNw5amQRIoK1dx5STAruNbgVct-PxbkFDXJVGCHD6h3Tiq-bHFCB2jzOxE_cQ
Fetching: 6 out of 87 - PUUID: 6IOjmETQ7eRL6-_sK6-aQ_bzrrSRUjvXGa6j_LeFH766XGdjJGywRvC-15L0ABnRKNQNFsWaYjzhpg
Fetching: 7 out of 87 - PUUID: WjFoz60M8Vg6I1xwAOCungN_iG5XjdI-NTNkx6qvxu4XH5QXjUdDWu5-jGJxKAwNZWcc-4zYch9DIw
Fetching: 8 out of 87 - PUUID: dWsPPisk499MJKFCQAFuqIFdU8Zd8hlT-EpGzHt1MJ_2hKDBFzXZ8842-Qs8jeWYxei3nT3tcvxNCg
Fetching: 9 out of 87 - PUUID: ouLXSMH1whxQZ9Lhf3gvcDYFKe43xuVgQcMBJrmmlr19c4Z_djiDBLcW_r6HPlHwR1ufvE8yQaKNiw
Fetching: 